Integración de la Inteligencia Artificial Mistral

Instanciación de la AI Mistral

In [19]:
import os
from dotenv import load_dotenv
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("Error con las variables de entorno.")

model = "mistral-large-latest"

client = MistralClient(api_key=api_key)


Prueba del chat

In [20]:
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content="Cuál es el significado de 'ubicuo'?")]
)

print(chat_response.choices[0].message.content)

La palabra "ubicuo" se utiliza para describir algo que está presente en todas partes o en muchos lugares al mismo tiempo. Proviene del latín "ubique", que significa "en todas partes". Por ejemplo, se puede decir que el internet es ubicuo porque está disponible en muchos lugares y es accesible desde diversas ubicaciones.


##### 1er paso: solicitud al sitio para saber si existe "robots.txt"

In [21]:
import requests

def get_robots_txt(url):
    if not url.endswith('/'):
        url += '/'
    
    robots_url = url + "robots.txt"
    
    response = requests.get(robots_url)
    
    if response.status_code == 200:
        return response.text
    else:
        return None

website_url = "https://radiocut.fm"
robots_txt = get_robots_txt(website_url)

if robots_txt:
    print("robots.txt content:")
    print(robots_txt)
else:
    print("robots.txt no encontrado")

robots.txt content:
User-agent: *
Crawl-delay: 30
Request-rate: 1/3
Sitemap: https://radiocut.fm/sitemap.xml
Disallow: /*/delete$




##### 2do paso: hacer la consulta al sitio. 
###### Ajustar el craw_delay y el request_rate con la librería "time"

In [22]:
import requests
import time

def fetch_url(url):
    response = requests.get(url)
    return response

# Recortes del día
urls = ["https://ar.radiocut.fm/search/?type=cut&page=4&created=1",
 "https://ar.radiocut.fm/search/?type=cut&page=4&created=2"]

crawl_delay = 30  # segundos
request_rate = 1 / 3  # 1 ssolicitud cada 3 segundos

docs = []
for url in urls:
    response = fetch_url(url)
    if response.status_code == 200:
        print("Fetched:", url)
        docs.append(response.text)

    else:
        print("Failed to fetch:", url)
    
    # Esperar de acuerdo al tiempo del crawl (rebaje)
    time.sleep(crawl_delay)
    

Fetched: https://ar.radiocut.fm/search/?type=cut&page=4&created=1
Fetched: https://ar.radiocut.fm/search/?type=cut&page=4&created=2


In [23]:
len(docs)

2

In [24]:
docs

['\n<!DOCTYPE html>\n<html lang="es">\n<head>\n<link rel="preconnect dns-prefetch" href="https://www.googletagmanager.com" />\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width,initial-scale=1.0" />\n<meta name="alexaVerifyID" content="VnZ_Et-Jpg6HoBQRuWYP1dYWaa0" />\n<meta property="fb:app_id" content="471603786214644" />\n<meta property="og:site_name" content="RadioCut" />\n<meta property="og:image" content="https://static.radiocut.com.ar/images/radio_cut_96.png" />\n<meta name="description" content="\n            \n            Escucha radios online de Argentina\n            \n          y programas de la radio FM, AM, Noticias, Música... Retrocede horas y días atrás y crea tus propios recortes de la radio.\n        ">\n<meta name="keywords" content="radios,radiodifusion,programa de radio,noticias,archivo de radios, podcast, deportes,musica,radiocut">\n<meta name="apple-itunes-app" content="app-id=1120326976">\n<meta name="google-play-app" content="app-id=com.l

##### 3er paso: scrapping o recauchute

In [25]:
from bs4 import BeautifulSoup

## Librerías FECHA
import re
from datetime import datetime, timedelta
import pytz
import locale

# Establece la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

soup = BeautifulSoup(docs[0], 'html.parser')

# Métodos FECHA
def clean_duration(duration_text):
    return duration_text.replace("Dur: ", "").strip()

def convert_duration_to_date(duration_text):
    pattern = re.compile(r'(\d+)\s*año|(\d+)\s*mes|(\d+)\s*semana|(\d+)\s*día|(\d+)\s*hora|(\d+)\s*minuto')
    matches = pattern.findall(duration_text)
    
    years = months = weeks = days = hours = minutes = 0
    for match in matches:
        if match[0]: years = int(match[0])
        if match[1]: months = int(match[1])
        if match[2]: weeks = int(match[2])
        if match[3]: days = int(match[3])
        if match[4]: hours = int(match[4])
        if match[5]: minutes = int(match[5])
    
    now = datetime.now(pytz.timezone('America/Argentina/Buenos_Aires'))
    time_delta = timedelta(days=(years * 365 + months * 30 + weeks * 7 + days), hours=hours, minutes=minutes)
    result_date = now - time_delta
    utc_dt = result_date.astimezone(pytz.utc)
    
    # Formatea el día y el mes manualmente para evitar ceros a la izquierda
    day = utc_dt.day
    month = utc_dt.strftime('%B')
    year = utc_dt.year
    time_str = utc_dt.strftime('%H:%M:%S')
    readable_date = f'{day} de {month} de {year} {time_str}'
    
    return readable_date

data_list = []

# Cucharada de sopa
containers = soup.find_all('div', class_='col-sm-10 col-xs-8')

for container in containers:
    # Extraer el título
    title_tag = container.find('a')
    title = title_tag.text.strip() if title_tag else None

    if title:  # Sólo sigue si hay título
        # Extrae la duración y la emprolija
        duration_tag = container.find('span', class_='text-muted')
        duration = clean_duration(duration_tag.text) if duration_tag else None

        # Extrae la descripción y la emprolija
        description_tag = container.find('p', class_='col-sm-12 description text-left')
        description = description_tag.text.strip() if description_tag else None

        # Extrae la fecha de creación del contenedor actual
        p_element = container.find_all('p')[-1]  # Asumiendo que la fecha está en el último <p> del contenedor
        first_text = p_element.text.strip().split('\n')[0].strip() if p_element else ""
        readable_date = convert_duration_to_date(first_text)
        
        # Crear un diccionario para almacenar la data
        data = {
            "title": title,
            "duration": duration,
            "description": description,
            "fecha": readable_date
        }
        
        if description:
            data_list.append(data)
            print(data)

{'title': 'Gabriela conversa con Florencia Aroldi sobre "Prestame tu sueño"', 'duration': '07:43', 'description': 'En la previa del estreno de la obra que escribió inspirada en la relación con su padre: el "flaco" Aroldi.', 'fecha': '3 de agosto de 2024 20:54:24'}
{'title': 'Martín Martinelli 4/8/2024', 'duration': '14:24', 'description': 'Entrevista en Sputnik a Martín Martinelli, analista internacional, a partir del asesinato de Ismail Haniyeh.', 'fecha': '2 de agosto de 2024 15:54:24'}
{'title': 'Belinetz va a llegar tu gol', 'duration': '00:08', 'description': 'Belinetz va a llevar tu gol', 'fecha': '4 de agosto de 2024 19:54:24'}
{'title': 'El Capital Financiero se quiere ir y el Club del Petróleo no entra si no devalúan (Horacio Rovelli)', 'duration': '24:00', 'description': 'Horacio Rovelli destaca la derecha se da el lujo de dirimir su interna, el Capital Financiero que se quiere ir y el Club del Petróleo. Viejas y nuevas formas de dominio.Escuchar las palabras de Rovelli en la

Obtengo la descripción con más caracteres como para saber

In [26]:
def find_max_description_length(data_list):
    max_length = 0
    dictDescLenMax = {}
    for item in data_list:
        description = item.get("description", "")
        current_length = len(description)
        currentMax = item
        if current_length > max_length:
            max_length = current_length
            dictDescLenMax = currentMax
    return dictDescLenMax

dictDescLenMax = find_max_description_length(data_list).get("description")
max_length = len(dictDescLenMax)
print(f"La descripción con longitud máxima tiene: {max_length} caracteres con espacios")
print(f"y es: {dictDescLenMax}")


La descripción con longitud máxima tiene: 1288 caracteres con espacios
y es: Horacio Rovelli destaca la pelea entre los grupos económicos financieros que aún están en pesos (muchas veces asociados, voluntariamente o no, al gran capital en la Argentina) y el Club del Petróleo. Ricardo Grosso y José María Cardo.Escuchar las palabras de Rovelli en la marcha de las Madres de Plaza de Mayo del jueves del 1/08/24.https://ar.radiocut.fm/audiocut/discurso-horacio-rovelli/Colonización.Viejas y nuevas formas de dominioLos ricos de este país (la vieja y la nueva casta) dividen sus negocios entre el carry trade y el sistema colonial tradicional, por supuesto que apuestan a ambos frentes; es más, la implosión de la deuda pública es funcional a la magna devaluación que necesitan para comprar por menos de la mitad lo que vale el doble.El problema es cuándo van a devaluar y el retraso fue que apostaron muchos dólares a comprar títulos en pesos y todos no se pueden ir.Mientras ellos dirimen sus cuentas

##### 4to paso: convertir el resultado, que es una lista de diccionarios, en un txt

In [27]:
def flatDictToTxt(dict_list, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for dictionary in dict_list:
            for key, value in dictionary.items():
                # Escribe cada llave y valor en el archivo
                file.write(f"{key}: {value} ")

file_path = 'output.txt'

# Escritura del archivo
flatDictToTxt(data_list, file_path)

##### 5to paso: cargar en la base de datos de vectores

###### Para probar: https://github.com/pgvector/pgvector

###### Para convertir el diccionario en un pdf o convertir un pdf a txt: https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/

###### Para usar FAISS Instalación de miniconda3 y: https://docs.conda.io/projects/conda/en/stable/user-guide/getting-started.html

In [28]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_mistralai import MistralAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("Error con las variables de entorno.")

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

mistral = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=api_key
)

# Specify the correct encoding
loader = TextLoader("output.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
texts = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(texts, mistral)

c:\Python311\Lib\site-packages\langchain_mistralai\embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


6to paso: preguntarle al chat

###### Fuente: https://colab.research.google.com/github/mistralai/cookbook/blob/main/mistral/rag/basic_RAG.ipynb#scrollTo=d67a4729-cd2f-47e7-a4f6-f84a5677414f

In [29]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Define una interfaz retriever (ahí está el context)
retriever = vectorstore.as_retriever()
# Instancia el Chat
model = ChatMistralAI(api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Responde la siguiente pregunta basándote únicamente en el contexto provisto:

<context>
{context}
</context>

Question: {input}""")

# Crea una cadena de retrieval para responder a la pregunta
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "¿De qué temas se habló?"})
print(response["answer"])

Based on the provided context, the following topics were discussed:

1. Gabriela's conversation with Florencia Aroldi about "Prestame tu sueño," an obra inspired by Gabriela's relationship with her father, the "flaco" Aroldi.
2. An interview with Martín Martinelli, an international analyst, discussing the assassination of Ismail Haniyeh.
3. A short clip about Belinetz, possibly a sports-related topic.
4. An analysis of the ongoing economic situation, colonization, and the struggle between different economic groups in Argentina by Horacio Rovelli.
5. A news update on the cancellation of more than 200 scientists' fellowships at CONICET due to budget cuts.
6. A radio broadcast of the soccer match between Unión and River Plate.
7. A continuation of the economic analysis and conflict between different economic groups in Argentina by Horacio Rovelli.
8. The impact of climate change on fertility, with a specific focus on how heatwaves reduce male fertility.

These topics cover various genres,